<a href="https://colab.research.google.com/github/michaelb3245/DSRP--MichaelBriglall/blob/main/Gallstone_Dataset_Michael_Briglall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
path = "/content/drive/MyDrive/Gallstone Dataset - Michael Briglall/dataset-uci (3).csv"
df = pd.read_csv(path)
df.head(5)

,Unnamed: 0,Gallstone Status,Age,Gender,Comorbidity,Coronary Artery Disease (CAD),Hypothyroidism,Hyperlipidemia,Diabetes Mellitus (DM),Height,...,High Density Lipoprotein (HDL),Triglyceride,Aspartat Aminotransferaz (AST),Alanin Aminotransferaz (ALT),Alkaline Phosphatase (ALP),Creatinine,Glomerular Filtration Rate (GFR),C-Reactive Protein (CRP),Hemoglobin (HGB),Vitamin D
0,1,0,50,0,0,0,0,0,0,185,...,40.0,134.0,20.0,22.0,87.0,0.82,112.47,0.0,16.0,33.0
1,2,0,47,0,1,0,0,0,0,176,...,43.0,103.0,14.0,13.0,46.0,0.87,107.10,0.0,14.4,25.0
2,3,0,61,0,0,0,0,0,0,171,...,43.0,69.0,18.0,14.0,66.0,1.25,65.51,0.0,16.2,30.2
3,4,0,41,0,0,0,0,0,0,168,...,59.0,53.0,20.0,12.0,34.0,1.02,94.10,0.0,15.4,35.4
4,5,0,42,0,0,0,0,0,0,178,...,30.0,326.0,27.0,54.0,71.0,0.82,112.47,0.0,16.8,40.6
